# Libraries and Installations

In [1]:
!pip install sagemaker-experiments
!pip install s3fs
!pip install matplotlib
!pip install seaborn
!pip install shap
!pip install smdebug


  Using cached sagemaker_experiments-0.1.30-py3-none-any.whl (42 kB)
     |████████████████████████████████| 356 kB 20.4 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 1.1 MB/s  eta 0:00:01
  Created wheel for shap: filename=shap-0.39.0-cp36-cp36m-linux_x86_64.whl size=419939 sha256=f4a15a973b6c926f71c53e1fcfdc198fec40a206734e635a235ace658a4d20df
  Stored in directory: /home/ec2-user/.cache/pip/wheels/6f/08/25/2992725334291786ea084e06cac493d93049b80e3470318a1b
Successfully built shap
     |████████████████████████████████| 263 kB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 16.5 MB/s eta 0:00:01


In [2]:
from io import StringIO
import numpy as np
import os
import pandas as pd
import boto3
import time
import s3fs
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import re
import shap
from scipy import stats
import copy

In [3]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.analytics import ExperimentAnalytics

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from sagemaker.sklearn.estimator import SKLearn
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig,CollectionConfig
from sagemaker.estimator import Estimator
from sagemaker.session import s3_input
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import Session

from smdebug.trials  import create_trial

[2021-04-29 13:00:19.999 ip-172-16-22-47:5486 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


# Configs

In [4]:
now = datetime.now()

current_time = now.strftime("%Y-%m-%d--%H-%M-%S")
print("current_time:",current_time)

sagemaker_session = sagemaker.Session()

role = 'arn:aws:iam::752400441523:role/Sagemaker_Access'

bucket = '1905-assignment2-sm'
prefix = 'Scikit-pre-model-Inference-Pipelines'

train_data = 's3://1905-assignment2-sm/housing/imput-datasets/train_data_without_header.csv'
test_data = 's3://1905-assignment2-sm/housing/imput-datasets/test_data_without_header.csv'

FRAMEWORK_VERSION = "0.23-1"
script_path = 'sklearn_pipeline.py'
dependency_path ='dependencies.py'

base_job_name = f"Builtin-XGB-algo-{current_time}"

output_data_prefix = f'housing/datasets/output/{base_job_name}'
data_output_path = f's3://{bucket}/{output_data_prefix}'

debug_prefix = f'housing/jobs/debug/{base_job_name}'
debug_path = f's3://{bucket}/{debug_prefix}'

experiment_name_prefix = "builtin-xgboost-track13"

current_time: 2021-04-29--13-00-54


In [5]:
train_data

's3://1905-assignment2-sm/housing/imput-datasets/train_data_without_header.csv'

In [6]:
role

'arn:aws:iam::752400441523:role/Sagemaker_Access'

# Batch transform

## Fit the train data

In [7]:
sklearn_preprocessor = SKLearn(
    entry_point = script_path,
    role = role,
    framework_version = FRAMEWORK_VERSION,
    train_instance_type =   "local" , #"ml.m5.xlarge",
    train_use_spot_instance = True,
    train_max_run = 600,
   # train_max_wait = 1200,
    dependencies = [dependency_path],
#    sagemaker_session = sagemaker_session
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.


In [8]:
role

'arn:aws:iam::752400441523:role/Sagemaker_Access'

In [9]:
sklearn_preprocessor.fit(
    inputs={'train':train_data},
    job_name=base_job_name)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: Builtin-XGB-algo-2021-04-29--13-00-54
INFO:sagemaker.local.local_session:Starting training job
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-cz8zr:
    command: train
    container_name: lfbblfgj6z-algo-1-cz8zr
    environment:
    - '[Masked]'
    - '[Masked]'
    image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-cz8zr
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmp_shdyfpa/algo-1-cz8zr/input:/opt/ml/input
    - /tmp/tmp_shdyfpa/algo-1-cz8zr/output/data:/opt/m

Creating lfbblfgj6z-algo-1-cz8zr ... 
Creating lfbblfgj6z-algo-1-cz8zr ... done
Attaching to lfbblfgj6z-algo-1-cz8zr
lfbblfgj6z-algo-1-cz8zr | 2021-04-29 13:02:17,515 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
lfbblfgj6z-algo-1-cz8zr | 2021-04-29 13:02:17,519 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
lfbblfgj6z-algo-1-cz8zr | 2021-04-29 13:02:17,532 sagemaker_sklearn_container.training INFO     Invoking user training script.
lfbblfgj6z-algo-1-cz8zr | 2021-04-29 13:02:18,004 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
lfbblfgj6z-algo-1-cz8zr | 2021-04-29 13:02:18,020 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
lfbblfgj6z-algo-1-cz8zr | 2021-04-29 13:02:18,036 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
lfbblfgj6z-algo-1-cz8zr | 2021-04-29 13:02:18,048 sagemaker-training-toolkit INFO     In

## Transform the training data

In [10]:
transformer = sklearn_preprocessor.transformer(
    instance_count=1,
    instance_type="local" , #'ml.m5.xlarge',
    assemble_with = 'Line',
    accept = 'text/csv',
    output_path=data_output_path)

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2021-04-29-13-02-56-967


In [11]:
transformer.transform(
    data=train_data,
    content_type="text/csv",
    job_name=base_job_name+'-train')

print("Waiting for transform job:" + transformer.latest_transform_job.job_name)
transformer.wait()

INFO:sagemaker:Creating transform job with name: Builtin-XGB-algo-2021-04-29--13-00-54-train
INFO:sagemaker.local.image:serving
INFO:sagemaker.local.image:creating hosting dir in /tmp/tmpwlm5rxgr
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-l2gni:
    command: serve
    container_name: a4cz7eypx6-algo-1-l2gni
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-l2gni
    ports:
    - 8080:8080
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmpifbpfaxk:/opt/ml/model
version: '2.3'

INFO:sagemaker.local.image:docker command: docker-compose -f /tmp/tmpwlm5

Attaching to a4cz7eypx6-algo-1-l2gni
a4cz7eypx6-algo-1-l2gni | 2021-04-29 13:03:05,052 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
a4cz7eypx6-algo-1-l2gni | 2021-04-29 13:03:05,056 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
a4cz7eypx6-algo-1-l2gni | 2021-04-29 13:03:05,057 INFO - sagemaker-containers - nginx config: 
a4cz7eypx6-algo-1-l2gni | worker_processes auto;
a4cz7eypx6-algo-1-l2gni | daemon off;
a4cz7eypx6-algo-1-l2gni | pid /tmp/nginx.pid;
a4cz7eypx6-algo-1-l2gni | error_log  /dev/stderr;
a4cz7eypx6-algo-1-l2gni | 
a4cz7eypx6-algo-1-l2gni | worker_rlimit_nofile 4096;
a4cz7eypx6-algo-1-l2gni | 
a4cz7eypx6-algo-1-l2gni | events {
a4cz7eypx6-algo-1-l2gni |   worker_connections 2048;
a4cz7eypx6-algo-1-l2gni | }
a4cz7eypx6-algo-1-l2gni | 
a4cz7eypx6-algo-1-l2gni | http {
a4cz7eypx6-algo-1-l2gni |   include /etc/nginx/mime.types;
a4cz7eypx6-algo-1-l2gni |   default_type application/octet-stream;
a4cz7eypx6-algo-1-l2gn

INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 10
INFO:sagemaker.local.entities:Container still not up, got: 502


a4cz7eypx6-algo-1-l2gni | 2021/04/29 13:03:07 [crit] 13#13: *1 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 172.18.0.1, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "localhost:8080"
a4cz7eypx6-algo-1-l2gni | 172.18.0.1 - - [29/Apr/2021:13:03:07 +0000] "GET /ping HTTP/1.1" 502 182 "-" "python-urllib3/1.26.4"
a4cz7eypx6-algo-1-l2gni | [2021-04-29 13:03:07 +0000] [30] [INFO] Starting gunicorn 20.0.4
a4cz7eypx6-algo-1-l2gni | [2021-04-29 13:03:07 +0000] [30] [INFO] Listening at: unix:/tmp/gunicorn.sock (30)
a4cz7eypx6-algo-1-l2gni | [2021-04-29 13:03:07 +0000] [30] [INFO] Using worker: gevent
a4cz7eypx6-algo-1-l2gni | [2021-04-29 13:03:07 +0000] [33] [INFO] Booting worker with pid: 33
a4cz7eypx6-algo-1-l2gni | [2021-04-29 13:03:07 +0000] [34] [INFO] Booting worker with pid: 34


INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 15


a4cz7eypx6-algo-1-l2gni | 2021-04-29 13:03:12,570 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
a4cz7eypx6-algo-1-l2gni | *****model function*****
a4cz7eypx6-algo-1-l2gni | /opt/ml/model
a4cz7eypx6-algo-1-l2gni | 172.18.0.1 - - [29/Apr/2021:13:03:13 +0000] "GET /ping HTTP/1.1" 200 0 "-" "python-urllib3/1.26.4"
a4cz7eypx6-algo-1-l2gni | 2021-04-29 13:03:13,240 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
a4cz7eypx6-algo-1-l2gni | *****model function*****
a4cz7eypx6-algo-1-l2gni | /opt/ml/model
a4cz7eypx6-algo-1-l2gni | 172.18.0.1 - - [29/Apr/2021:13:03:13 +0000] "GET /execution-parameters HTTP/1.1" 404 232 "-" "python-urllib3/1.26.4"
a4cz7eypx6-algo-1-l2gni | *****predict function*****
a4cz7eypx6-algo-1-l2gni | /miniconda3/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:440: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
a

In [12]:
preprocessed_train_data = transformer.output_path

In [13]:
preprocessed_train_data

's3://1905-assignment2-sm/housing/datasets/output/Builtin-XGB-algo-2021-04-29--13-00-54'

## Transform the test data

In [14]:
transformer.transform(
    data=test_data,
    content_type="text/csv",
    job_name=base_job_name+"-test")

print("Waiting for transform job:" + transformer.latest_transform_job.job_name)
transformer.wait()

INFO:sagemaker:Creating transform job with name: Builtin-XGB-algo-2021-04-29--13-00-54-test
INFO:sagemaker.local.image:serving
INFO:sagemaker.local.image:creating hosting dir in /tmp/tmpead3ctgb
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-x2wfe:
    command: serve
    container_name: tt3pb5kzs3-algo-1-x2wfe
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-x2wfe
    ports:
    - 8080:8080
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmpgnnpgrr3:/opt/ml/model
version: '2.3'

INFO:sagemaker.local.image:docker command: docker-compose -f /tmp/tmpead3c

Attaching to tt3pb5kzs3-algo-1-x2wfe
tt3pb5kzs3-algo-1-x2wfe | 2021-04-29 13:03:58,828 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
tt3pb5kzs3-algo-1-x2wfe | 2021-04-29 13:03:58,831 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
tt3pb5kzs3-algo-1-x2wfe | 2021-04-29 13:03:58,832 INFO - sagemaker-containers - nginx config: 
tt3pb5kzs3-algo-1-x2wfe | worker_processes auto;
tt3pb5kzs3-algo-1-x2wfe | daemon off;
tt3pb5kzs3-algo-1-x2wfe | pid /tmp/nginx.pid;
tt3pb5kzs3-algo-1-x2wfe | error_log  /dev/stderr;
tt3pb5kzs3-algo-1-x2wfe | 
tt3pb5kzs3-algo-1-x2wfe | worker_rlimit_nofile 4096;
tt3pb5kzs3-algo-1-x2wfe | 
tt3pb5kzs3-algo-1-x2wfe | events {
tt3pb5kzs3-algo-1-x2wfe |   worker_connections 2048;
tt3pb5kzs3-algo-1-x2wfe | }
tt3pb5kzs3-algo-1-x2wfe | 
tt3pb5kzs3-algo-1-x2wfe | http {
tt3pb5kzs3-algo-1-x2wfe |   include /etc/nginx/mime.types;
tt3pb5kzs3-algo-1-x2wfe |   default_type application/octet-stream;
tt3pb5kzs3-algo-1-x2wf

INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 10


tt3pb5kzs3-algo-1-x2wfe | 2021-04-29 13:04:02,101 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
tt3pb5kzs3-algo-1-x2wfe | *****model function*****
tt3pb5kzs3-algo-1-x2wfe | /opt/ml/model
tt3pb5kzs3-algo-1-x2wfe | 172.18.0.1 - - [29/Apr/2021:13:04:02 +0000] "GET /ping HTTP/1.1" 200 0 "-" "python-urllib3/1.26.4"
tt3pb5kzs3-algo-1-x2wfe | 172.18.0.1 - - [29/Apr/2021:13:04:02 +0000] "GET /execution-parameters HTTP/1.1" 404 232 "-" "python-urllib3/1.26.4"
tt3pb5kzs3-algo-1-x2wfe | 2021-04-29 13:04:02,867 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
tt3pb5kzs3-algo-1-x2wfe | *****model function*****
tt3pb5kzs3-algo-1-x2wfe | /opt/ml/model
tt3pb5kzs3-algo-1-x2wfe | *****predict function*****
tt3pb5kzs3-algo-1-x2wfe | /miniconda3/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:440: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
t

In [15]:
preprocessed_test_data = transformer.output_path

In [16]:
f'{output_data_prefix}'

'housing/datasets/output/Builtin-XGB-algo-2021-04-29--13-00-54'

## Real time Prediction using endpoint

In [24]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime
from sagemaker.estimator import Estimator
from sagemaker import PipelineModel

timestamp_prefix = current_time

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()
scikit_learn_inferencee_model.env = {"SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT":"text/csv"}
model_containers = [scikit_learn_inferencee_model]

model_name = 'inference-pipeline-' + current_time
endpoint_name = 'inference-pipeline-ep-' + current_time


In [25]:
endpoint_name

'inference-pipeline-ep-2021-04-29--13-00-54'

In [36]:
predictor = sm_model.deploy(
    initial_instance_count=1,
                           instance_type="local_gpu" , #'ml.m5.xlarge',
                           endpoint_name=endpoint_name,
                           #data_capture_config=data_capture_config
                           )

INFO:sagemaker:Creating model with name: inference-pipeline-2021-04-29--13-00-54


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: 1 validation error detected: Value 'local_gpu' at 'productionVariants.1.member.instanceType' failed to satisfy constraint: Member must satisfy enum value set: [ml.r5d.12xlarge, ml.r5.12xlarge, ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.r5d.24xlarge, ml.r5.24xlarge, ml.p3.16xlarge, ml.m5d.xlarge, ml.m5.large, ml.t2.xlarge, ml.p2.16xlarge, ml.m5d.12xlarge, ml.inf1.2xlarge, ml.m5d.24xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c4.4xlarge, ml.inf1.6xlarge, ml.c5d.2xlarge, ml.c5.4xlarge, ml.g4dn.xlarge, ml.g4dn.12xlarge, ml.c5d.4xlarge, ml.g4dn.2xlarge, ml.c4.8xlarge, ml.c4.large, ml.c5d.xlarge, ml.c5.large, ml.g4dn.4xlarge, ml.c5.9xlarge, ml.g4dn.16xlarge, ml.c5d.large, ml.c5.xlarge, ml.c5d.9xlarge, ml.c4.xlarge, ml.inf1.xlarge, ml.g4dn.8xlarge, ml.inf1.24xlarge, ml.m5d.2xlarge, ml.t2.2xlarge, ml.c5d.18xlarge, ml.m5d.4xlarge, ml.t2.medium, ml.c5.18xlarge, ml.r5d.2xlarge, ml.r5.2xlarge, ml

In [23]:




from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

payload = "-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY"

predictor = Predictor(
        endpoint_name = endpoint_name,
       # sagemaker_session = sagemaker_session,
        serializer = CSVSerializer(),
        deserializer = JSONDeserializer(),
    )


INFO:sagemaker:Creating model with name: inference-pipeline-2021-04-29--13-00-54


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: 1 validation error detected: Value 'local' at 'productionVariants.1.member.instanceType' failed to satisfy constraint: Member must satisfy enum value set: [ml.r5d.12xlarge, ml.r5.12xlarge, ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.r5d.24xlarge, ml.r5.24xlarge, ml.p3.16xlarge, ml.m5d.xlarge, ml.m5.large, ml.t2.xlarge, ml.p2.16xlarge, ml.m5d.12xlarge, ml.inf1.2xlarge, ml.m5d.24xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c4.4xlarge, ml.inf1.6xlarge, ml.c5d.2xlarge, ml.c5.4xlarge, ml.g4dn.xlarge, ml.g4dn.12xlarge, ml.c5d.4xlarge, ml.g4dn.2xlarge, ml.c4.8xlarge, ml.c4.large, ml.c5d.xlarge, ml.c5.large, ml.g4dn.4xlarge, ml.c5.9xlarge, ml.g4dn.16xlarge, ml.c5d.large, ml.c5.xlarge, ml.c5d.9xlarge, ml.c4.xlarge, ml.inf1.xlarge, ml.g4dn.8xlarge, ml.inf1.24xlarge, ml.m5d.2xlarge, ml.t2.2xlarge, ml.c5d.18xlarge, ml.m5d.4xlarge, ml.t2.medium, ml.c5.18xlarge, ml.r5d.2xlarge, ml.r5.2xlarge, ml.p3.

In [38]:
print(predictor.predict(data=payload))

{'instances': [{'features': [-1.3358644914093034, 1.0625114171921215, 0.9843617822955569, -0.7963234446647113, -0.9664904061190807, -0.9606541571538452, -0.9683302007899453, 2.3372729061643134, 0.6152810290729377, -0.07388883558315307, -1.073845989834207, 0.0, 0.0, 0.0, 1.0, 0.0]}]}


In [39]:
#Delete the endpoint
sm_client = sagemaker_session.boto_session.client('sagemaker')
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '215f8680-8a28-4feb-9008-2fead898ae7a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '215f8680-8a28-4feb-9008-2fead898ae7a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 29 Apr 2021 07:52:29 GMT'},
  'RetryAttempts': 0}}